# Découpage de mp3 en fonction d'un CSV avec les débuts-fins

In [1]:
from pydub import AudioSegment
from lxml import etree
import glob,codecs,datetime,time,re,mp3_tagger,unidecode
import pandas as pd
from cellbell import ding

In [2]:
repBase="/Users/gilles/Desktop/Rugby/"
Mp3Fr=repBase+"Japon-Fidji-FR.mp3"
Mp3Jp=repBase+"Japon-Fidji-JP.mp3"
soundFr = AudioSegment.from_mp3(Mp3Fr)
soundJp = AudioSegment.from_mp3(Mp3Jp)

In [3]:
fDecoupages=repBase+"ActionsProem_Japon-Fidji.csv"
dfDecoupages=pd.read_csv(filepath_or_buffer=fDecoupages,sep="\t",encoding="utf8")
# renommer les colonnes
dfDecoupages.columns="evenement jDeb jFin fDeb fFin minutes avantage".split(" ")
# supprimer la première ligne (sous-titres)
dfDecoupages=dfDecoupages.iloc[1:]

## Fixing time problems

In [4]:
def correctTime(t):
    lT=t
    # si le temps n'est que sur deux positions, ajouter des heures vides
    m=re.match(ur"^\d\d:\d\d(\.\d+)*$",t)
    if m:
        lT="00:"+t
    else:
        # si les secondes sont vides, il est possible qu'il faille décaler H > M > S et ajouter des heures vides
        m=re.match(ur"^(\d\d):(\d\d):(\d\d)(\.\d+)*$",lT)
        lSec=m.group(3)
        lHour=m.group(1)
        if int(lSec)==0:
            print "suspicious time",t
            lT="00:"+m.group(1)+":"+m.group(2)
            print "corrected to",lT
    return lT       


for ix,row in dfDecoupages.iloc[:7].iterrows():
    row["fDeb"]=correctTime(row["fDeb"])
    row["fFin"]=correctTime(row["fFin"])
    row["jDeb"]=correctTime(row["jDeb"])
    row["jFin"]=correctTime(row["jFin"])
    

suspicious time 24:24:00
corrected to 00:24:24
suspicious time 24:34:00
corrected to 00:24:34
suspicious time 28:04:00
corrected to 00:28:04
suspicious time 28:14:00
corrected to 00:28:14
suspicious time 37:42:00
corrected to 00:37:42
suspicious time 37:49:00
corrected to 00:37:49
suspicious time 41:25:00
corrected to 00:41:25
suspicious time 41:38:00
corrected to 00:41:38
suspicious time 44:29:00
corrected to 00:44:29
suspicious time 44:45:00
corrected to 00:44:45
suspicious time 48:10:00
corrected to 00:48:10
suspicious time 48:21:00
corrected to 00:48:21
suspicious time 45:22:00
corrected to 00:45:22
suspicious time 45:29:00
corrected to 00:45:29
suspicious time 49:06:00
corrected to 00:49:06
suspicious time 49:15:00
corrected to 00:49:15
suspicious time 49:55:00
corrected to 00:49:55
suspicious time 50:11:00
corrected to 00:50:11
suspicious time 53:37:00
corrected to 00:53:37
suspicious time 53:55:00
corrected to 00:53:55


In [5]:
dfDecoupages

,evenement,jDeb,jFin,fDeb,fFin,minutes,avantage
1,Pénalité,00:12:08,00:12:23,00:08:22,00:08:42,4',Fidji
2,Pénalité,00:28:04,00:28:14,00:24:24,00:24:34,18',Japon
3,Pénalité,00:41:25,00:41:38,00:37:42,00:37:49,30',Japon
4,Essai,00:48:10,00:48:21,00:44:29,00:44:45,35',Fidji
5,Transformation,00:49:06,00:49:15,00:45:22,00:45:29,37',Fidji
6,Pénalité,00:53:37,00:53:55,00:49:55,00:50:11,40',Japon
7,Pénalité,01:02:44,01:02:53,01:03:59,01:04:12,43',Japon
8,Essai,01:09:14,01:09:32,01:10:34,01:10:51,49',Fidji
9,Transformation,01:10:50,01:10:57,01:12:12,01:12:28,50',Fidji
10,Essai,01:11:43,01:12:07,01:13:02,01:13:12,51',Japon


In [6]:
def timeDiff(fin,debut="00:00:00"):
    lDeb=datetime.datetime.strptime(debut,'%H:%M:%S')
    lFin=datetime.datetime.strptime(fin,'%H:%M:%S')
    return abs((lFin-lDeb).seconds)


In [7]:
timeDiff("00:00:01")

1

In [8]:
def makeExtrait(debut,fin,langue,son):
    fDeb=timeDiff(debut)
    fFin=timeDiff(fin)
    fExtrait=repBase+"extrait-%s-%02d.mp3"%(langue,ix)
    print fExtrait
    extrait=son[int(fDeb*1000):int(fFin*1000)]
    extrait.export(fExtrait, format="mp3")
    mp3=mp3_tagger.MP3File(fExtrait)
    mp3.set_version(mp3_tagger.VERSION_1)
    mp3.artist=avantage
    mp3.album=unidecode.unidecode(evenement)
    mp3.song=debut+" - "+fin
    mp3.comment=""
    mp3.track="%02d"%ix
#     mp3.genre=1
    mp3.year="2007"
    mp3.save()
    print mp3.get_tags()
    

for ix,row in dfDecoupages.iterrows():
    fDeb=timeDiff(row["fDeb"])
    fFin=timeDiff(row["fFin"])
    jDeb=timeDiff(row["jDeb"])
    jFin=timeDiff(row["jFin"])
    avantage=row["avantage"]
    evenement=row["evenement"]
    makeExtrait(row["fDeb"],row["fFin"],"Fr",soundFr)
    makeExtrait(row["jDeb"],row["jFin"],"Jp",soundJp)
    

/Users/gilles/Desktop/Rugby/extrait-Fr-01.mp3
{'album': u'Penalite', 'comment': None, 'track': 1, 'song': u'00:08:22 - 00:08:42', 'genre': 'Leftfield', 'artist': u'Fidji', 'year': u'2007'}
/Users/gilles/Desktop/Rugby/extrait-Jp-01.mp3
{'album': u'Penalite', 'comment': None, 'track': 1, 'song': u'00:12:08 - 00:12:23', 'genre': 'Bebop', 'artist': u'Fidji', 'year': u'2007'}
/Users/gilles/Desktop/Rugby/extrait-Fr-02.mp3
{'album': u'Penalite', 'comment': None, 'track': 2, 'song': u'00:24:24 - 00:24:34', 'genre': 'Leftfield', 'artist': u'Japon', 'year': u'2007'}
/Users/gilles/Desktop/Rugby/extrait-Jp-02.mp3
{'album': u'Penalite', 'comment': None, 'track': 2, 'song': u'00:28:04 - 00:28:14', 'genre': 'Leftfield', 'artist': u'Japon', 'year': u'2007'}
/Users/gilles/Desktop/Rugby/extrait-Fr-03.mp3
{'album': u'Penalite', 'comment': None, 'track': 3, 'song': u'00:37:42 - 00:37:49', 'genre': 'Bebop', 'artist': u'Japon', 'year': u'2007'}
/Users/gilles/Desktop/Rugby/extrait-Jp-03.mp3
{'album': u'Penal

In [ ]:
ding()